In [ ]:
import numpy as np
import osmnx as ox
import utm

from geo.math import num_haversine, vec_haversine
from geo.road import RoadNetwork, download_road_network
from geo.trajectory import load_trajectory_points

In [ ]:
road_network = download_road_network("Ann Arbor, Michigan, USA")
# road_network = download_road_network("Michigan, USA")

## Matching Edges with OSMnx

We start by using OSMnx's own functions to find edge matches to locations sampled from the EVED. The first thing we need to do is to project the whole road network to the local UTM projection.

In [ ]:
network_utm = ox.projection.project_graph(road_network)

The above operation converts latitudes and longitudes to a local UTM projection that works as a Cartesian plane in meters. The advantage of such projection is that you can directly use your knowledge of vectors to work out distances and other planar geometry calculations. The disadvantage is that you always need to perform the conversion before using OSMnx's functions. When converting from (_latitude_, _longitude_) to UTM, we get a converted coordinate pair (_x_, _y_) aptly named (_easting_, _northing_), along with the UTM number and letter codes. We might need these later to convert (_easting_, _northing_) back to (_latitude_, _longitude_).

In [ ]:
easting, northing, zone_num, zone_ltr = utm.from_latlon(42.287702, -83.707775)

We can now call OSMnx's `nearest_edges` [function](https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.distance.nearest_edges) to determine the closest edges.

In [ ]:
edge_id = ox.distance.nearest_edges(network_utm, easting, northing)

In [ ]:
edge_id

The first two numbers in the tuple identify the edge's nodes, and we can query it from the network using the following:

In [ ]:
network_utm[edge_id[0]][edge_id[1]][0]

In [ ]:
network_utm[edge_id[1]][edge_id[0]][0]

In [ ]:
trajectory = load_trajectory_points(4, unique=True)
raw_lists = map(list, zip(*trajectory))
latitudes, longitudes, bearings = map(np.array, raw_lists)

In [ ]:
eastings, northings, zone_num, zone_ltr = utm.from_latlon(latitudes, longitudes)

In [ ]:
len(eastings), len(northings)

**OSMnx**

In [ ]:
%%timeit
ox.distance.nearest_edges(network_utm, eastings, northings)

**Fitness Ratio**

In [ ]:
%%timeit
rn = RoadNetwork(road_network)
cache = dict()
for p in trajectory:
    if p not in cache:
        edge = rn.get_matching_edge(*p)
        cache[p] = edge

**Distance**

In [ ]:
%%timeit
rn = RoadNetwork(road_network)
cache = dict()
for p in trajectory:
    if p not in cache:
        edge = rn.get_nearest_edge(*p)
        cache[p] = edge

In [ ]:
edges_0 = ox.distance.nearest_edges(network_utm, eastings, northings)

In [ ]:
rn = RoadNetwork(road_network)
edges_1 = []
cache = dict()
for p in trajectory:
    if p not in cache:
        edge = rn.get_matching_edge(*p)
        cache[p] = edge
    edges_1.append(cache[p])

In [ ]:
rn = RoadNetwork(road_network)
edges_2 = []
cache = dict()
for p in trajectory:
    if p not in cache:
        edge = rn.get_nearest_edge(*p)
        cache[p] = edge
    edges_2.append(cache[p])

In [ ]:
e0 = [(e[0], e[1]) for e in edges_0 if e is not None]
e1 = [(e[0], e[1]) for e in edges_1 if e is not None]
e2 = [(e[0], e[1]) for e in edges_2 if e is not None]

In [ ]:
len(e0), len(e1), len(e2)

In [ ]:
max([road_network[e[0]][e[1]][0]["length"] for e in road_network.edges if "length" in road_network[e[0]][e[1]][0]])

In [ ]:
ds = []
for e in road_network.edges:
    n0 = road_network.nodes[e[0]]
    n1 = road_network.nodes[e[1]]
    ds.append(num_haversine(n0['y'], n0['x'], n1['y'], n1['x']))

In [ ]:
max(ds)